In [1]:
# draft_1
ROW = 0 # indexing purposes -> Python start the index from 0
COLUMN = 1 # indexing purpose

class SampleEnvironment():
    def __init__(self, n_rows=1, n_cols=5):
        self.time = 0
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.pedestrian_destination = self.create_pedestrian_destination()
        self.pedestrian = self.create_pedestrian()
        self.grid = self.create_grid(self.pedestrian, n_rows, n_cols)

    @staticmethod
    def create_pedestrian_destination(): # will later be randomized
        destination_row = 0
        destination_col = 4
        return (destination_row, destination_col)

    @staticmethod
    def create_pedestrian(): # will later be randomized and increased the quantity
        pedestrian_row = 0
        pedestrian_col = 0
        return (pedestrian_row, pedestrian_col)

    @staticmethod
    def create_grid(pedestrian, n_rows, n_cols):
        grid = []
        for idx_row in range(n_rows):
            column=  []
            for idx_column in range(n_cols):
                column.append(0)
            grid.append(column)
        grid[pedestrian[ROW]][pedestrian[COLUMN]] = 1
        return grid

    @staticmethod    
    def update_pedestrian_location(pedestrian, n_rows, n_cols): # will later have multiple ways to "walk"
        pedestrian_row = pedestrian[ROW]
        pedestrian_col = pedestrian[COLUMN] + 1
        return(pedestrian_row, pedestrian_col)

    def reset(self):
        self.time = 0
        self.pedestrian = self.create_pedestrian()
        self.grid = self.create_grid(self.pedestrian, self.n_rows, self.n_cols)
        return self.grid

    def step(self):
        self.time = self.time + 1
        self.pedestrian = self.update_pedestrian_location(self.pedestrian, self.n_rows, self.n_cols)
        self.grid = self.create_grid(self.pedestrian, self.n_rows, self.n_cols)
        newobs = self.grid
        if self.pedestrian == self.pedestrian_destination:
            done = True
        else:
            done = False
        info = {
            'goal' : self.pedestrian_destination,
            'time' : self.time
        }
        return newobs, done, info

    def render(self): # will later change into grid
        for rows in self.grid:
            print(rows)

In [2]:
env = SampleEnvironment()

In [3]:
obs = env.reset()
print("obs : {}".format(obs))

obs : [[1, 0, 0, 0, 0]]


In [4]:
env.render()

[1, 0, 0, 0, 0]


In [5]:
for i in range(4):
    newobs, done, info = env.step()
    print('newobs : {}'.format(newobs))
    print('done : {}'.format(done))
    print('info : {}'.format(info))
    i = i+1

newobs : [[0, 1, 0, 0, 0]]
done : False
info : {'goal': (0, 4), 'time': 1}
newobs : [[0, 0, 1, 0, 0]]
done : False
info : {'goal': (0, 4), 'time': 2}
newobs : [[0, 0, 0, 1, 0]]
done : False
info : {'goal': (0, 4), 'time': 3}
newobs : [[0, 0, 0, 0, 1]]
done : True
info : {'goal': (0, 4), 'time': 4}


In [75]:
# draft_2
# import:
import random
from itertools import product
random.seed(1)

# indexing:
STARTING = 0
DESTINATION = 1
ROW = 0
COLUMN = 1

# variables:
list_position = [(0,0), (0,4)]
num_of_pedestrian = 2
n_rows = 1
n_cols = 5

# draft_2_pedestrian
class Pedestrian():
 
    def __init__(self, list_position, num_of_pedestrian):
        self.list_position = list_position
        self.list_sd = self.create_starting_destination(list_position)
        self.list_pedestrian_sd = self.create_pedestrian_sd(num_of_pedestrian)
        # create_speed

    staticmethod
    def create_starting_destination(list_position):
        starting_point = list_position # since list mentioned in the init method, can we put self.list_position?
        destination_point = list_position
        list_sd = []
        for starting, destination in product(starting_point, destination_point):
            if starting != destination:
                list_sd.append((starting, destination))
        return list_sd # return a list of tuple

    @staticmethod
    def create_pedestrian_sd(num_of_pedestrian):
        pedestrian_s = []
        pedestrian_d = []
        list_pedestrian_sd = [pedestrian_s, pedestrian_d]
        for idx_pedestrian_starting in range(num_of_pedestrian):
            pedestrian_sd = random.choice(Pedestrian.create_starting_destination(list_position))
            pedestrian_s.append(pedestrian_sd[STARTING])
            pedestrian_d.append(pedestrian_sd[DESTINATION])
        return list_pedestrian_sd

    @staticmethod
    def create_speed(list_speed): # not used yet
        return list_speed
    
    @staticmethod
    def create_pathway(list_pedestrian_s, pedestrian_loc):
        for idx_pedestrian in range(num_of_pedestrian):
            pedestrian_s = list_pedestrian_s[idx_pedestrian]
            if pedestrian_s == (0,0): # generalize this
                pedestrian_row = pedestrian_loc[ROW]
                pedestrian_col = pedestrian_loc[COLUMN] + 1
            else:
                pedestrian_row = pedestrian_loc[ROW]
                pedestrian_col = pedestrian_loc[COLUMN] - 1
            pedestrian_loc = [pedestrian_row, pedestrian_col]
            return pedestrian_loc

list_pedestrian_sd = Pedestrian.create_pedestrian_sd(num_of_pedestrian)
list_pedestrian_s = list_pedestrian_sd[STARTING]
list_pedestrian_d = list_pedestrian_sd[DESTINATION]

# draft_2_environment

class SampleEnvironment():
    
    def __init__(self, n_rows, n_cols, list_pedestrian_sd, list_pedestrian_s):
        self.time = 0
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.pedestrian_s = list_pedestrian_sd[STARTING]
        self.pedestrian_d = list_pedestrian_sd[DESTINATION]
        self.grid = self.create_location_grid(list_pedestrian_s, n_rows, n_cols)

    @staticmethod
    def create_pedestrian(num_of_pedestrian):
        pedestrian_loc = []
        for idx_pedestrian in range(num_of_pedestrian):
            pedestrian_s = list_pedestrian_s[idx_pedestrian]
            pedestrian_row = pedestrian_s[ROW]
            pedestrian_col = pedestrian_s[COLUMN]
            pedestrian_loc.append((pedestrian_row, pedestrian_col))
        return pedestrian_loc

    @staticmethod
    def create_location_grid(list_pedestrian_s, n_rows, n_cols):
        grid = []
        for idx_row in range(n_rows):
            row = []
            for idx_column in range(n_cols):
                row.append(0)
            grid.append(row)
        for idx_pedestrian in range(num_of_pedestrian):
            pedestrian_s = list_pedestrian_s[idx_pedestrian]
            grid[pedestrian_s[ROW]][pedestrian_s[COLUMN]] = 1
        return grid

    @staticmethod
    def update_pedestrian_location(list_pedestrian_s, pedestrian_loc):
        pedestrian_row = Pedestrian.create_pathway(list_pedestrian_s, pedestrian_loc)[ROW]
        pedestrian_col = Pedestrian.create_pathway(list_pedestrian_s, pedestrian_loc)[COLUMN]
        return pedestrian_row, pedestrian_col

    def reset(self):
        self.time = 0
        self.pedestrian_loc = self.create_pedestrian(num_of_pedestrian)
        self.grid = self.create_location_grid(self.pedestrian_s, self.n_rows, self.n_cols)
        return self.grid

    def step(self):
        self.time = self.time + 1
        self.pedestrian_loc = self.update_pedestrian_location(list_pedestrian_s, self.pedestrian_loc)
        self.grid = self.create_location_grid(list_pedestrian_s, self.n_rows, self.n_cols)
        newobs = self.grid
        for idx_pedestrian in range(num_of_pedestrian):
            if self.pedestrian_loc[idx_pedestrian] == list_pedestrian_d[idx_pedestrian]:
                done = True
            else:
                done = False
            info = {
            'goal' : list_pedestrian_d[idx_pedestrian],
            'time' : self.time
            }
        return newobs, done, info


TypeError: Pedestrian.create_starting_destination() missing 1 required positional argument: 'list_position'

In [71]:
env = SampleEnvironment(n_rows, n_cols, list_pedestrian_sd, list_pedestrian_s)

In [72]:
obs = env.reset()
print("obs : {}".format(obs))

obs : [[1, 0, 0, 0, 0]]


In [55]:
print(Pedestrian.list_pedestrian_sd)

AttributeError: type object 'Pedestrian' has no attribute 'list_pedestrian_sd'

In [73]:
def create_starting_destination(list_position):
        starting_point = list_position
        destination_point = list_position
        list_sd = []
        for starting, destination in product(starting_point, destination_point):
            if starting != destination:
                list_sd.append((starting, destination))
        return list_sd

print(create_starting_destination(list_position=[(0,0), (0,4)]))


[((0, 0), (0, 4)), ((0, 4), (0, 0))]


In [77]:
def create_pedestrian_sd(num_of_pedestrian):
        pedestrian_s = []
        pedestrian_d = []
        list_pedestrian_sd = [pedestrian_s, pedestrian_d]
        for idx_pedestrian_starting in range(num_of_pedestrian):
            pedestrian_sd = random.choice(Pedestrian.create_starting_destination(list_position))
            pedestrian_s.append(pedestrian_sd[STARTING])
            pedestrian_d.append(pedestrian_sd[DESTINATION])
        return list_pedestrian_sd
        
print(Pedestrian.create_starting_destination(list_position=[(0,0), (0,4)]))
print(create_pedestrian_sd(num_of_pedestrian=3))

TypeError: Pedestrian.create_starting_destination() missing 1 required positional argument: 'self'